In [3]:
import pandas as pd
import tabula

In [11]:
def convertidor_int_a_formato_sigfe(numero):
    numero = str(numero)
    primer_bloque = numero[0: 2]
    segundo_bloque = numero[2: 4]
    tercer_bloque = numero[4: 7]

    nuevo_formato = '.'.join([primer_bloque, segundo_bloque, tercer_bloque])

    if len(numero) >= 10:
        cuarto_bloque = numero[7: 10]
        nuevo_formato = '.'.join([nuevo_formato, cuarto_bloque])

        if len(numero) == 12:
            quinto_bloque = numero[10: 12]
            nuevo_formato = '.'.join([nuevo_formato, quinto_bloque])
    
    return nuevo_formato

def formato_chequeador(numero_formateado):
    excepciones = ['22.01.001', '22.02.002', '22.04.004.001', '22.04.005', '22.06.002', '22.09.005',
                   '22.11.002', '22.12.999.009']
    
    for excepcion in excepciones:
        if excepcion in numero_formateado:
            numero_formateado = excepcion
            break
    
    return numero_formateado

In [12]:
df_suministros = pd.read_excel('CRUDO_SUMINISTROS.xlsx')
df_adquisiciones = pd.read_excel('CRUDO_SIGFE.xlsx', header = 3)

In [28]:
df_adquisiciones_codigos_unicos = df_adquisiciones.drop_duplicates(subset = 'Código')
df_adquisiciones_codigos_unicos['Items'] = df_adquisiciones_codigos_unicos['Items'].apply(convertidor_int_a_formato_sigfe)
df_adquisiciones_codigos_unicos['Cod. SIGFE para Matchear'] = df_adquisiciones_codigos_unicos['Items'].apply(formato_chequeador)
df_adquisiciones_codigos_unicos = df_adquisiciones_codigos_unicos.rename(columns = {'Items': 'Cod. SIGFE', 'Código': 'Cod. Bodega'})

df_sigcom_sigfe = pd.read_excel('asociacion_sigcom_sigfe_modificada.xlsx')
df_sigcom_sigfe = df_sigcom_sigfe.rename(columns = {'Código': 'Cod. SIGCOM', 'Cod. SIGFE': 'Cod. SIGFE para Matchear'})

left_df = df_adquisiciones_codigos_unicos.iloc[:, [0, 1, 2, 3, 4, 5, -1]]
right_df = df_sigcom_sigfe

df_adquisiciones_sigcom = pd.merge(left_df, right_df, how = 'left', on = 'Cod. SIGFE para Matchear')
df_adquisiciones_sigcom = df_adquisiciones_sigcom.drop(columns = 'Cod. SIGFE para Matchear')
df_adquisiciones_sigcom = df_adquisiciones_sigcom.set_index('Cod. Bodega', drop = True)
df_adquisiciones_sigcom = df_adquisiciones_sigcom[['Descripción', 'Cod. SIGFE', 'Cod. SIGCOM', 'Nº',
 'Familia',
 'Nombre Items',
 'Item en SIGCOM',
 'Item SIGFE',
 'Gastos SIGCOM']]

df_adquisiciones_sigcom.to_json('asociacion_bodega_sigcom.json', orient = 'index', indent = 1, force_ascii = False)

C:\Users\gmolina\AppData\Local\Temp\ipykernel_24044\1437378731.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_adquisiciones_codigos_unicos['Items'] = df_adquisiciones_codigos_unicos['Items'].apply(convertidor_int_a_formato_sigfe)
C:\Users\gmolina\AppData\Local\Temp\ipykernel_24044\1437378731.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_adquisiciones_codigos_unicos['Cod. SIGFE para Matchear'] = df_adquisiciones_codigos_unicos['Items'].apply(formato_chequeador)


In [29]:
df_adquisiciones_sigcom.to_excel('Bodega_SIGCOM.xlsx')